# Hello everyone.

I completed the day 1, first LLM Experiment moments ago and found it really awesome. After the challenge was done, I wanted to chip in my two cents by making a PDF summarizer, basing myself on the code for the Website Summarizer. I want to share it in this contribution!
### To consider:
* To extract the contents of PDF files, I used the PyPDF2 library, which doesn't come with the default configuration of the virtual environment. To remedy the situation, you need to follow the steps:
  1. Shut down Anaconda. Running `CTRL-C` in the Anaconda terminal should achieve this.
  2. Run the following command, `pip install PyPDF2 --user`
  3. Restart Jupyter lab with `jupyter lab`
* To find PDF files online, you can add `filetype:url` on your browser query, i.e. searching the following can give you PDF files to add as input: `AI Engineering prompts filetype:pdf`!

Without further ado, here's the PDF Summarizer!

In [ ]:
### Prerequisite - Install below package
#--> !pip install PyPDF2 --user

In [ ]:
# Import statements
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from io import BytesIO
from PyPDF2 import PdfReader
import logging

In [ ]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [60]:
# Step 1: Defince calss for article pdf document

class Article:
    def __init__(self, url):
        # Configure logging
        logging.basicConfig(level=logging.ERROR)

        # Comprehensive headers to mimic browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'application/pdf,text/html,application/xhtml+xml,application/xml',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': self._extract_base_url(url)
        }

        try:
            # Enhanced download attempt with multiple strategies
            response = self._download_pdf(url, headers)
            
            # Process PDF
            pdf_bytes = BytesIO(response.content)
            reader = PdfReader(pdf_bytes)
            
            # Efficient text extraction
            self.text = " ".join(page.extract_text() for page in reader.pages)
            
            # Safe metadata extraction
            self.title = (reader.metadata or {}).get("/Title", "No title found") or "No title found"

        except Exception as error:
            logging.error(f"PDF Processing Error: {error}")
            self.text = "No text found"
            self.title = "No title found"

    def _download_pdf(self, url, headers):
        """
        Multiple download strategies to handle different access scenarios
        """
        try:
            # Primary download attempt
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            return response

        except requests.RequestException as primary_error:
            logging.warning(f"Primary download failed: {primary_error}")
            
            try:
                # Fallback strategy: Disable SSL verification
                response = requests.get(url, headers=headers, verify=False, timeout=10)
                response.raise_for_status()
                return response
            
            except requests.RequestException as fallback_error:
                logging.error(f"All download attempts failed: {fallback_error}")
                raise

    def _extract_base_url(self, url):
        """
        Extract base URL for Referer header
        """
        from urllib.parse import urlparse
        parsed_url = urlparse(url)
        return f"{parsed_url.scheme}://{parsed_url.netloc}"

#####user_prompt_input = input("Explain how you would like me to summarise the document? \n")

In [61]:
# General Research Analyst
#### Step 1: Create your prompts

def craft_user_prompt(article):
    user_prompt = f"You are looking at a document titled {article.title}\n Based on the body of the document, provide an impactful summary. The body of the article is as follows."
    user_prompt += article.text
    return user_prompt

### Step 2: Make the messages list
def craft_messages(article):
    system_prompt = '''You are a research assistant specialising in processing documents. You have been tasked with analysing the contents of the articel and call out the important points as bullets and highlights. \
    Ignore text that doesn't belong to the article, like headers or navigation related text. Respond in markdown. Structure your text in the form of question/answer.'''
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": craft_user_prompt(article)}
    ]

In [62]:
# Step 3: Call OpenAI
def summarize(url):
    article = Article(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = craft_messages(article)
    )
    return response.choices[0].message.content

# Results below:

In [63]:
# Step 4: Print the result of an example pdf
article_url=input('Please input the pdf URL: \n')
summary = summarize(article_url)
display(Markdown(summary))

Please input the pdf URL: 
 https://www.binasss.sa.cr/bibliotecas/bhm/ago22/32.pdf


## Summary of **Bronchiectasis — A Clinical Review**

### What is bronchiectasis?
- Bronchiectasis is a clinical syndrome characterized by:
  - Chronic cough 
  - Sputum production
  - Abnormal thickening and dilation of bronchial walls, visible on lung imaging.

### Historical Background
- First reported by René Laënnec in 1819.
- Radiographic characteristics understood more clearly since the 1950s.

### Current Trends
- Significantly increasing incidence and prevalence over the last 20 years.
- Diagnosed across a wide age range, with geographic variability in prevalence (as high as 1.5% in some populations).
- Improved recognition, partly due to advanced imaging technologies like CT scans.
    
### Clinical Presentation
- Symptoms include chronic productive cough, chest pain, and shortness of breath.
- Often misdiagnosed as chronic bronchitis or asthma, leading to delayed diagnosis.
- Intermittent exacerbations are common, defined by a worsening of cough and sputum characteristics for 48 hours or more.

### Diagnostic Approaches
- High-resolution CT scans are essential for diagnosis, seeking specific airway abnormalities.
- A systematic evaluation to identify underlying causes (such as congenital disorders or autoimmune diseases) is critical.

### Pathobiology
- Multiple factors lead to airway dysfunction, inflammatory responses, and cyclical deterioration.
- Neutrophils and neutrophil elastase play significant roles in disease progression and exacerbations.

### Microbiological Features
- **Common pathogens**:
  - Pseudomonas aeruginosa (notably linked to exacerbations and worse prognosis)
  - Staphylococcus aureus and other bacteria.
- Non-tuberculous mycobacterial infections are reported increasingly.

### Treatment Strategies
- Emphasizes education about the disease and its management.
- Goals include symptom management, quality of life improvement, and reduction of exacerbation frequency.
- Options include:
  - Airway-clearance therapies
  - Antibiotics (macrolides or inhaled antibiotics)
  - Addressing any treatable underlying conditions.

### Prognostic Indicators
- Disease severity scores like the Bronchiectasis Severity Index and FACED scale are used for predicting outcomes.

### Future Directions
- Emphasis on research into novel therapies targeting inflamatory pathways and innovative diagnostic tools.

### Conclusion
- Bronchiectasis represents a complex clinical challenge that requires a multidisciplinary approach for effective management and treatment improvements. Further understanding of the disease mechanisms, enhancing diagnostic processes, and tailored therapies are vital for future advancements in the care of patients.

# Financial Report Analyst
#### Step 1: Create your prompts

def craft_user_prompt(article):
    user_prompt = f"You are looking at a document titled {article.title}\n Based on the body of the document, provide an impactful summary. Call out important points as bullets and highlughts . The body of the article is as follows."
    user_prompt += article.text
    return user_prompt

#### Step 2: Make the messages list
def craft_messages(article):
    system_prompt = '''You are a financial report research assistant that specialises in analysing the contents of a financial report and provide summary for investors to \ 
    help adjust investment strategies. Call out important points as bullets and highlights. \
    Ignore text that doesn't belong to the article, like headers or navigation related text. Respond in markdown. Structure your text in the form of question/answer.'''
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": craft_user_prompt(article)}
    ]

# Medical Report Analyst
#### Step 1: Create your prompts

def craft_user_prompt(article):
    user_prompt = f"You are looking at a research article titled {article.title}\n Based on the body of the article, Summarise the research article while highliting, Introduction, the problem, possible solution and conclusion. The body of the article is as follows."
    user_prompt += article.text
    return user_prompt

#### Step 2: Make the messages list
def craft_messages(article):
    system_prompt = "You are a medical research assistant that analyses the contents of a research article and provides a summary while calling out important points as bullets and quotes. \
                Ignore text that doesn't belong to the article, like headers or navigation related text. Respond in markdown. Structure your text in the form of question/answer."
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": craft_user_prompt(article)}
    ]

# QA Format
#### Step 1: Create your prompts

def craft_user_prompt(article):
    user_prompt = f"You are looking at a research article titled {article.title}\n Based on the body of the article, how are micro RNAs produced in the cell? State the function of the proteins \
    involved. The body of the article is as follows."
    user_prompt += article.text
    return user_prompt

#### Step 2: Make the messages list
def craft_messages(article):
    system_prompt = "You are an assistant that analyses the contents of a research article and provide answers to the question asked by the user in 250 words or less. \
                Ignore text that doesn't belong to the article, like headers or navigation related text. Respond in markdown. Structure your text in the form of question/answer."
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": craft_user_prompt(article)}
    ]